In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.svm import SVC,SVR
from sklearn.linear_model import SGDRegressor,SGDClassifier,RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,NearestNeighbors
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from catboost import CatBoost,CatBoostClassifier,CatBoostRegressor

In [160]:
data = pd.read_csv('./cleaned_data.csv')
data.sample(frac=1)
test_data = pd.read_csv('./cleaned_test.csv')
sample_sub = pd.read_csv('./gender_submission.csv')

## pd.get_dummies() vs OneHotEncoding() and MinMaxScalar() vs OneHotEncoding() and StandordScalar()

### pd.get_dummies()

In [161]:
pd_get_dummied_mean = []

In [162]:
# ## 2.45810593901
# for _ in range(25):
#   pd_get_dummied_data = pd.get_dummies(data)
#   X = pd_get_dummied_data.drop('Survived',axis=1)
#   y = pd_get_dummied_data['Survived']
#   X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
#   model = CatBoostClassifier(task_type='GPU')
#   model.fit(X_train,y_train,verbose=1)
#   preds = np.round(model.predict(X_test))
#   print(accuracy_score(y_test,preds))
#   pd_get_dummied_mean.append(accuracy_score(y_test,preds))
# print(np.mean(pd_get_dummied_mean))

### OneHotEncoding() + MinMaxScalar()

In [163]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [164]:
# ## 2.44382022472
# for _ in range(25):
#   X = data.drop('Survived',axis=1)
#   y = data['Survived']
#   X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
#   one_hot_encoding_and_min_max_scalar_mean = []
#   from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler
#   from sklearn.compose import make_column_transformer
#   ct = make_column_transformer(
#       (OneHotEncoder(),['Name','Sex','Ticket','Cabin','Embarked']),
#       (MinMaxScaler(),['PassengerId','Pclass','SibSp','Parch','Fare']),
#   )
#   ct.fit(X)
#   X_train = ct.transform(X_train)
#   X_test = ct.transform(X_test)
#   X_train = X_train.toarray()
#   X_test = X_test.toarray()
#   model = CatBoostClassifier(task_type='GPU')
#   model.fit(X_train,y_train,verbose=1)
#   preds = np.round(model.predict(X_test))
#   print(accuracy_score(y_test,preds))
#   one_hot_encoding_and_min_max_scalar_mean.append(accuracy_score(y_test,preds))
# print(np.mean(one_hot_encoding_and_min_max_scalar_mean))

### OneHotEncoding() + StandordScalar()

In [165]:
# ## 1.60112359551
# for _ in range(10):
#   X = data.drop('Survived',axis=1)
#   y = data['Survived']
#   X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
#   one_hot_encoding_and_standord_scalar_mean = []
#   from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler
#   from sklearn.compose import make_column_transformer
#   ct = make_column_transformer(
#       (OneHotEncoder(),['Name','Sex','Ticket','Cabin','Embarked']),
#       (StandardScaler(),['PassengerId','Pclass','SibSp','Parch','Fare']),
#   )
#   ct.fit(X)
#   X_train = ct.transform(X_train)
#   X_test = ct.transform(X_test)
#   X_train = X_train.toarray()
#   X_test = X_test.toarray()
#   model = CatBoostClassifier(task_type='GPU')
#   model.fit(X_train,y_train,verbose=1)
#   preds = np.round(model.predict(X_test))
#   print(accuracy_score(y_test,preds))
#   one_hot_encoding_and_standord_scalar_mean.append(accuracy_score(y_test,preds))
# print(np.mean(one_hot_encoding_and_standord_scalar_mean))

In [166]:
## 2.44382022472
X = data.drop('Survived',axis=1)
y = data['Survived']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
one_hot_encoding_and_min_max_scalar_mean = []
from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler,RobustScaler
from sklearn.compose import make_column_transformer
ct = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'),['Name','Sex','Ticket','Cabin','Embarked']),
    (RobustScaler(),['PassengerId','Pclass','SibSp','Parch','Fare']),
)
ct.fit(X)
X_train = ct.transform(X_train)
X_test = ct.transform(X_test)
X_train = X_train.toarray()
X_test = X_test.toarray()

## Modelling

In [156]:
from sklearn.svm import SVC,SVR
from sklearn.linear_model import SGDRegressor,SGDClassifier,RidgeClassifier,LogisticRegression
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,NearestNeighbors
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from catboost import CatBoost,CatBoostClassifier,CatBoostRegressor
models = [
    ['SVC',SVC()],
    ['SGDClassifier',SGDClassifier()],
    ['RidgeClassifier',RidgeClassifier()],
    ['KNeighborsClassifier',KNeighborsClassifier()],
    ['GaussianNB',GaussianNB()],
    ['RandomForestClassifier',RandomForestClassifier()],
    ['CatBoost',CatBoost()],
    ['CatBoostClassifier',CatBoostClassifier()],
    ['RandomForestClassifier2',RandomForestClassifier(n_estimators=100, oob_score = True, random_state = 1)],
    ['LogisticRegression',LogisticRegression(solver='liblinear')],
]
maxes = []
for _ in range(10):
  results = {}
  for model in models:
      print(model)
      model_name = model[0]
      model = model[1]
      model.fit(X_train,y_train)
      try:
          accuracy = model.score(X_test,y_test)
          results[model_name] = accuracy
      except:
          preds = model.predict(X_test)
          preds = np.round(preds)
          accuracy = accuracy_score(y_test,preds)
          results[model_name] = accuracy
      print(f'{model_name} : {accuracy}')
  print(results)
  maxes.append(max(results, key=results.get))

['SVC', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)]


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC : 0.6292134831460674
['SGDClassifier', SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)]
SGDClassifier : 0.7415730337078652
['RidgeClassifier', RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)]
RidgeClassifier : 0.7865168539325843
['KNeighborsClassifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')]


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


KNeighborsClassifier : 0.7752808988764045
['GaussianNB', GaussianNB(priors=None, var_smoothing=1e-09)]
GaussianNB : 0.4550561797752809
['RandomForestClassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)]
RandomForestClassifier : 0.8033707865168539
['CatBoost', <catboost.core.CatBoost object at 0x7fa7e2ec37b8>]


...

KNeighborsClassifier : 0.7752808988764045
['GaussianNB', GaussianNB(priors=None, var_smoothing=1e-09)]
GaussianNB : 0.4550561797752809
['RandomForestClassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)]
RandomForestClassifier : 0.7752808988764045
['CatBoost', <catboost.core.CatBoost object at 0x7fa7e2ec37b8>]
Learning rate set to 0.036731
0:	learn: 0.4785750	total: 1.5ms	remaining: 1.5s
1:	learn: 0.4722068	total: 4.87ms	remaining: 2.43s
2:	learn: 0.4658255	total: 7.75ms	remaining: 2.58s
3:	learn: 0.4599295	total: 10.7ms	remaining: 2.67s
4:	learn: 0.4542070	total: 13.7ms	remaining: 2.72s
5:	learn: 0.4496518	to

/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC : 0.6292134831460674
['SGDClassifier', SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)]
SGDClassifier : 0.7134831460674157
['RidgeClassifier', RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)]
RidgeClassifier : 0.7865168539325843
['KNeighborsClassifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')]


/home/indika/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


KNeighborsClassifier : 0.7752808988764045
['GaussianNB', GaussianNB(priors=None, var_smoothing=1e-09)]
GaussianNB : 0.4550561797752809
['RandomForestClassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)]
RandomForestClassifier : 0.8033707865168539
['CatBoost', <catboost.core.CatBoost object at 0x7fa7e2ec37b8>]
Learning rate set to 0.036731
0:	learn: 0.4785750	total: 1.54ms	remaining: 1.54s
1:	learn: 0.4722068	total: 3.67ms	remaining: 1.83s
2:	learn: 0.4658255	total: 5.73ms	remaining: 1.9s
3:	learn: 0.4599295	total: 7.98ms	remaining: 1.99s
4:	learn: 0.4542070	total: 10.8ms	remaining: 2.15s
5:	learn: 0.4496518	t

In [157]:
maxes

['RandomForestClassifier',
 'RandomForestClassifier2',
 'RandomForestClassifier',
 'RandomForestClassifier',
 'RandomForestClassifier2',
 'RandomForestClassifier',
 'RandomForestClassifier',
 'RandomForestClassifier2',
 'RandomForestClassifier2',
 'RandomForestClassifier']

In [158]:
from collections import Counter
occurence_count = Counter(maxes)
print(occurence_count.most_common(1)[0][0])

RandomForestClassifier


In [173]:
# grid = {
#     "n_estimators":[100,250,500,1000],
#     "max_depth":[None,5,10,round(12.5)],
#     "max_features":["auto","sqrt",'log2'],
#     "min_samples_split":[2,5],
#     "min_samples_leaf":[2,5],
#     'bootstrap':[False,True],
#     'warm_start':[False,True]
# }
# model = RandomForestClassifier()
# gs_model = GridSearchCV(model,param_grid=grid,cv=round(2.5),verbose=2)
# gs_model.fit(X_train,y_train)
# print(f'{gs_model.score(X_test,y_test)}')

Fitting 2 folds for each of 768 candidates, totalling 1536 fits
[CV] bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100, warm_start=False 
[CV]  bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100, warm_start=False, total=   0.1s
[CV] bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100, warm_start=False 
[CV]  bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100, warm_start=False, total=   0.1s
[CV] bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100, warm_start=True 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100, warm_start=True, total=   0.1s
[CV] bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100, warm_start=True 
[CV]  bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100, warm_start=True, total=   0.1s
[CV] bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=250, warm_start=False 
[CV]  bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=250, warm_start=False, total=   0.2s
[CV] bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=250, warm_start=False 
[CV]  bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=250, warm_start=False, total=   0.2s
[CV] 

[Parallel(n_jobs=1)]: Done 1536 out of 1536 | elapsed:  8.7min finished


In [174]:
# grid = {
#     'alpha':[1,round(2.5),5,round(7.5),10,round(12.5),25,50,75,100],
#     'fit_intercept':[True,False],
#     'normalize':[False,True],
#     'copy_X':[False,True],
#     'max_iter':[None,1,round(2.5)],
#     'solver':['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
# }
# model = RidgeClassifier()
# gs_model = GridSearchCV(model,param_grid=grid,cv=round(2.5),verbose=2)
# gs_model.fit(X_train,y_train)
# print(f'{gs_model.score(X_test,y_test)}')

In [175]:
# grid = {
#     'n_neighbors':[round(2.5),5,round(7.5),10,round(12.5),25,50,75,100],
#     'weights':['distance','uniform'],
#     'algorithm':['auto', 'ball_tree','kd_tree','brute'],
#     'leaf_size':[round(12.5),25,50,75,100],
# }
# model = KNeighborsClassifier()
# gs_model = GridSearchCV(model,param_grid=grid,cv=round(2.5),verbose=2)
# gs_model.fit(X_train,y_train)
# print(f'{gs_model.score(X_test,y_test)}')

In [176]:
gs_model.best_estimator_ 

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [177]:
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.50000,0,0,330911,7.8292,40,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00000,1,0,363272,7.0000,40,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.00000,0,0,240276,9.6875,40,Q
3,895,3,"Wirz, Mr. Albert",male,27.00000,0,0,315154,8.6625,40,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00000,1,1,3101298,12.2875,40,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,30.27259,0,0,A.5. 3236,8.0500,40,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.00000,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,40,S
416,1308,3,"Ware, Mr. Frederick",male,30.27259,0,0,359309,8.0500,40,S


In [178]:
preds = gs_model.predict(ct.transform(test_data).toarray())

In [179]:
submission = pd.DataFrame({'PassengerId':test_data['PassengerId'],'Survived':preds})

In [180]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,0


In [181]:
submission.to_csv('./submission-5.csv',index=False)